# Libraries

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv3D,Conv2D, MaxPooling2D,TimeDistributed,LSTM,ConvLSTM2D
from keras.utils import np_utils

# Functions

In [ ]:
# Loading data

def load_data(x):

  data_set = "/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/"
  sx_path = data_set + x
  with open(sx_path, 'rb') as file:
    sx_data = pickle.load(file, encoding='latin1')
  c_ax=sx_data['signal']['chest']['ACC'][0:,0]
  c_ay=sx_data['signal']['chest']['ACC'][0:,1]
  c_az=sx_data['signal']['chest']['ACC'][0:,2]
  c_ecg=sx_data['signal']['chest']['ECG'][:,0]
  c_emg=sx_data['signal']['chest']['EMG'][:,0]
  c_eda=sx_data['signal']['chest']['EDA'][:,0]
  c_temp=sx_data['signal']['chest']['Temp'][:,0]
  c_resp=sx_data['signal']['chest']['Resp'][:,0]
  w_label=sx_data['label']
  numpy_data=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
  numpy_data=numpy_data.T
  chest_df = pd.DataFrame(data=numpy_data, 
                           columns=["c_ax", "c_ay", "c_az", "c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 
  chest_df['w_label'] = chest_df['w_label'].astype('int')
  df = chest_df.loc[chest_df['w_label'].isin([1,2,3])]
  return df



In [ ]:

def window(data, size, shift):
  
  limit = 3500  # window size
  w = []   # windows
  w_count = 0 #no of windows
  j= 0
  for i in range(len(data)):
      res = data.iloc[j:size,:]
      if len(res.index) != limit:
         break
      else:
         res= res.T
         w.append(res)
         j = j + shift
         size = shift + size
         w_count = w_count + 1
  
  return w , w_count


In [ ]:
# For making the data ready to be fed into the model

def data_ready(x_data , y_no, category):

  x = np.array(x_data)
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)

  return x,y 

# s17 - Test set

In [ ]:
s17_path = "S17/S17.pkl"
s17 = load_data(s17_path)
print(s17.head())
print()
print('Shape:')
print(s17.shape)
print()
print('dtypes:')
print(s17.dtypes)
print()
print('Value Counts:')
print(s17['w_label'].value_counts())
print()
s17_final = s17.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s17_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
64564  0.7838 -0.4066 -0.3706 -0.014648  ...  5.543137  32.567444  2.757263        1
64565  0.7842 -0.4040 -0.3678 -0.001236  ...  5.542755  32.567444  2.742004        1
64566  0.7864 -0.4056 -0.3688  0.007278  ...  5.556107  32.561493  2.603149        1
64567  0.7876 -0.4078 -0.3682  0.009201  ...  5.558014  32.559998  2.749634        1
64568  0.7870 -0.4066 -0.3726  0.007141  ...  5.564117  32.558502  2.760315        1

[5 rows x 9 columns]

Shape:
(1593200, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826700
2    506100
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
64564 -0.014648 -0.001968  5.543137  32.567444  2.757263        1
64565 -0.001236 -0.005539  5.542755  32.5

In [ ]:
print('Baseline:')
s17_base = s17_final.loc[s17_final['w_label'].isin([1])]
print(s17_base.shape)
s17_basex = s17_base.drop('w_label' ,axis = 1)
print(s17_basex.shape)
print()
print('Stress:')
s17_stress = s17_final.loc[s17_final['w_label'].isin([2])]
print(s17_stress.shape)
s17_stressx = s17_stress.drop('w_label',axis = 1)
print(s17_stressx.shape)
print()
print('Amusement:')
s17_amuse = s17_final.loc[s17_final['w_label'].isin([3])]
print(s17_amuse.shape)
s17_amusex = s17_amuse.drop('w_label',axis =1)
print(s17_amusex.shape)

Baseline:
(826700, 6)
(826700, 5)

Stress:
(506100, 6)
(506100, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s17_basexw, count = window(s17_basex , 3500 ,1400)
print(count)
print(len(s17_basexw))

s17_basexwa , s17_basey  = data_ready(s17_basexw , count , 1)
print(s17_basexwa.shape)
print(s17_basey.shape)

print()
print('Stress:')
s17_stressxw, count = window(s17_stressx , 3500 ,1400)
print(count)
print(len(s17_stressxw))
s17_stressxwa , s17_stressy  = data_ready(s17_stressxw , count , 2)
print(s17_stressxwa.shape)
print(s17_stressy.shape)

print()
print('Amusement:')
s17_amusexw, count = window(s17_amusex , 3500 ,1400)
print(count)
print(len(s17_amusexw))
s17_amusexwa , s17_amusey  = data_ready(s17_amusexw , count , 3)
print(s17_amusexwa.shape)
print(s17_amusey.shape)

Creating windows:

Baseline:
589
589
(589, 5, 3500)
(589,)

Stress:
360
360
(360, 5, 3500)
(360,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s17 = np.concatenate( (s17_basexwa, s17_stressxwa, s17_amusexwa) , axis = 0)
print(len(x_s17))
print(x_s17.shape)
s17_x =  x_s17.reshape(x_s17.shape[0], 5, 3500, 1)
print(s17_x.shape)

1133
(1133, 5, 3500)
(1133, 5, 3500, 1)


In [ ]:
y_s17 = np.concatenate( (s17_basey, s17_stressy, s17_amusey) , axis = 0)
s17_y = pd.DataFrame(y_s17)

print(s17_y.shape)
print()
print(s17_y.columns)
print(s17_y.value_counts())
print()
print(s17_y.head())

(1133, 1)

RangeIndex(start=0, stop=1, step=1)
1    589
2    360
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


In [ ]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( s17_x , s17_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:100])

(1076, 5, 3500, 1)
(57, 5, 3500, 1)
(1076, 1)
(57, 1)
      0
695   2
484   1
615   2
452   1
698   2
43    1
299   1
910   2
1067  3
683   2
716   2
852   2
67    1
173   1
244   1
326   1
274   1
1014  3
318   1
637   2
11    1
323   1
56    1
1111  3
895   2
953   3
784   2
1103  3
537   1
668   2
225   1
559   1
450   1
898   2
880   2
934   2
904   2
524   1
638   2
547   1
125   1
684   2
22    1
567   1
403   1
568   1
60    1
358   1
196   1
222   1
672   2
654   2
834   2
525   1
857   2
109   1
131   1


In [ ]:
xtest = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtest.npy", xtest)
print(xtest.shape)
ytest = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytest.npy", ytest)
print(ytest.shape)

(1133, 5, 3500, 1)
(1133, 1)
